This notebook analyses the differences between the sexes by age in Ireland, from the next dataset: cso-populationbyage.csv.

- Weighted mean age (by sex).
- The difference between the sexes by age.

In [14]:
import pandas as pd

In [15]:
FILENAME = 'cso-populationbyage.csv'
DATADIR = './'
FULLPATH = DATADIR + FILENAME

# read the CSV file into a DataFrame
df = pd.read_csv(FULLPATH)
print(df.head(3))  # print the first 3 rows to verify


  Statistic Label  CensusYear         Sex Single Year of Age  \
0      Population        2022  Both sexes           All ages   
1      Population        2022  Both sexes           All ages   
2      Population        2022  Both sexes           All ages   

  Administrative Counties    UNIT    VALUE  
0                 Ireland  Number  5149139  
1   Carlow County Council  Number    61968  
2     Dublin City Council  Number   592713  


In [16]:
#Create a new CSV file with the cleaned data
#df.to_csv("population_foranalysis.csv")

Drop columns that we don't need.

In [17]:
drop_col_list = ['Statistic Label', 'CensusYear', 'Sex', 'UNIT']
df.drop(columns=drop_col_list, inplace=True) # Inplace=True to modify the DataFrame directly
print(df.head(3))  # print the first 3 rows to verify

  Single Year of Age Administrative Counties    VALUE
0           All ages                 Ireland  5149139
1           All ages   Carlow County Council    61968
2           All ages     Dublin City Council   592713


Having words in the age field, such as "All ages," "under 1 year," "years and over," and "years," can be problematic when analyzing the data. Therefore, we will remove these words and keep only the age as a number.

First step is remove the "All ages" text.

In [18]:
df = df[df["Single Year of Age"] != "All ages"]
print(df.head(3))  # print the first 3 rows to verify

   Single Year of Age Administrative Counties  VALUE
32       Under 1 year                 Ireland  57796
33       Under 1 year   Carlow County Council    699
34       Under 1 year     Dublin City Council   6213


Second step is change the column using find and replace. We will replace everything that says "Under 1 year" for "0" consider the minors have less than 1 year.

In [19]:
df["Single Year of Age"] = df["Single Year of Age"].replace("Under 1 year", "0")
print(df.head(5))  # print the first 5 rows to verify

   Single Year of Age                Administrative Counties  VALUE
32                  0                                Ireland  57796
33                  0                  Carlow County Council    699
34                  0                    Dublin City Council   6213
35                  0  Dún Laoghaire Rathdown County Council   2457
36                  0                  Fingal County Council   4009


Third step is change the text that is repeated regulary in the data set "years". To avoid do this one by one: '2 years' '3 years', we can use regular expressions, that they help to change/delete large amounts of data that are repeated in the data set.

Source: https://www.w3schools.com/python/python_regex.asp

In [20]:
# Regular expression to remove 'years' and 'year' from the 'Single Year of Age' column
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'\D', '', regex=True) # add r to indicate raw string
print(df.tail(5))  # print the last 5 rows to verify

     Single Year of Age   Administrative Counties  VALUE
3259                100  Roscommon County Council     12
3260                100      Sligo County Council     12
3261                100      Cavan County Council     18
3262                100    Donegal County Council     33
3263                100   Monaghan County Council      8


In [21]:
#Create a new CSV file with the cleaned data
df.to_csv("population_for_analysis.csv")

In [22]:
print(df.info())  # print DataFrame info to verify data types

<class 'pandas.core.frame.DataFrame'>
Index: 3232 entries, 32 to 3263
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Single Year of Age       3232 non-null   object
 1   Administrative Counties  3232 non-null   object
 2   VALUE                    3232 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 101.0+ KB
None


As we can see the Single Year of age column is an object and we want to have this values as a integer. We can change this using 'type' to 'int 64' 

In [23]:
df["Single Year of Age"] = df["Single Year of Age"].astype('int64') # Convert to integer

In [24]:
print(df.info())  # print DataFrame info to verify data types

<class 'pandas.core.frame.DataFrame'>
Index: 3232 entries, 32 to 3263
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Single Year of Age       3232 non-null   int64 
 1   Administrative Counties  3232 non-null   object
 2   VALUE                    3232 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 101.0+ KB
None


## Pivot Tables
We can use pivot tables to summarize, analyze, and explore large datasets in a organized way.
And avoid certain issues when weuse cross tables. 


Source: https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

In [27]:
df_analysis = pd.pivot_table(df, 'VALUE', 'Single Year of Age', 'Administrative Counties')
print(df_analysis.head(10))  # print the first 10 rows to verify
df_analysis.to_csv("population_for_analysis.csv")

Administrative Counties  Carlow County Council  Cavan County Council  \
Single Year of Age                                                     
0                                        699.0                1006.0   
1                                        649.0                1004.0   
2                                        689.0                1061.0   
3                                        754.0                1085.0   
4                                        745.0                1047.0   
5                                        749.0                1143.0   
6                                        750.0                1176.0   
7                                        889.0                1162.0   
8                                        819.0                1217.0   
9                                        882.0                1289.0   

Administrative Counties  Clare County Council  Cork City Council  \
Single Year of Age                                                 